## Assignment 1
# Group 13
Mathieu Mailhot - Isabel Lougheed - Frank-Lucas Pantazis

Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

In [2]:
"""
- CKD: 28 numerical features, 1 target binary classification variable ("Normal" / "battery")
- Battery: 32 real-valued features, 2 classes ("Normal" / "Defective")
"""

# load data sets

# Calculate cross entropy or/ Information Gain for all the data without the threshold

# statistical analysis on the datasets

# - normalize

# models: all features, selective features based on statistical analysis (dropping features)


'\n- CKD: 28 numerical features, 1 target binary classification variable ("Normal" / "battery")\n- Battery: 32 real-valued features, 2 classes ("Normal" / "Defective")\n'

Statistics

In [3]:
# load data sets
df_CKD = pd.read_csv("CKD.csv")
df_battery = pd.read_csv("Battery_Dataset.csv")

# Convert "CKD" to 1 and "Normal" to 0
df_CKD["label"] = df_CKD["label"].replace({"CKD": 1, "Normal": 0})
# Convert "Defective" to 1 and "Normal" to 0
df_battery["label"] = df_battery["label"].replace({"Defective": 1, "Normal": 0})

# Convert to a numpy array
CKD_data = df_CKD.to_numpy()
battery_data = df_battery.to_numpy()

C:\Users\issyl\AppData\Local\Temp\ipykernel_30448\3014589073.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_CKD["label"] = df_CKD["label"].replace({"CKD": 1, "Normal": 0})
C:\Users\issyl\AppData\Local\Temp\ipykernel_30448\3014589073.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_battery["label"] = df_battery["label"].replace({"Defective": 1, "Normal": 0})


In [143]:
# Statistical Analysis Block

# Class for the analysis
class Stat_analysis:
    def __init__(self, data, name, save_folder):
        self.data = data
        self.name = name
        self.save_folder = save_folder
        self.feature_distribution()
        self.class_distrubution()

    # Function to create a distribution for each feature
    def feature_distribution(self):
        for i in range(self.data.shape[1] - 2): # remove 1 and last column as we do not need them for the distribution of the features
            feature_num = i + 1

            plt.hist(self.data[:,feature_num], bins=20, edgecolor="black")
            plt.xlabel("Value")
            plt.ylabel("Frequency")
            plt.title(f"{self.name} Distribution of Feature {feature_num}")

            filename = os.path.join(self.save_folder, f"{self.name}_feature{feature_num}_distribution.png")
            plt.savefig(filename, dpi=300, bbox_inches="tight")
            plt.close()
    
    # Function to create a distribution for the class
    def class_distrubution(self):
        my_bins = [-0.5, 0.5, 1.5]
        class_0 = "Normal"
        if self.name == "CKD":
            class_1 = "CKD"
        else:
            class_1 = "Defective"

        plt.hist(self.data[:,self.data.shape[1] - 1], bins=my_bins, edgecolor="black", align="mid", rwidth=0.6)
        plt.xticks([0, 1], [class_0, class_1])
        plt.xlabel("Category")
        plt.ylabel("Frequency")
        plt.title(f"{self.name} Distribution of Class")

        filename = os.path.join(self.save_folder, f"{self.name}_class_distribution.png")
        plt.savefig(filename, dpi=300, bbox_inches="tight")
        plt.close()

# Perform the stastical analysis
CKD_stat = Stat_analysis(CKD_data, "CKD", "CKD_distribution")
battery_stat = Stat_analysis(battery_data, "Battery", "Battery_distribution")




In [4]:
class Model:
    def __init__(self,  dataframe, data_array, description:str = None):
        self.max_iteration = 10000
        self.tolerance = 10**-9
        self.step_size = 0.001

        self.dataframe = dataframe
        self.data_array = data_array
        self.description = description
        
        self.accuracy_arr = 0
        self.avg_accuracy = 0
        
        self.weigth_arr = 0
        self.avg_weigth = 0
    
    def normalize(self, normalize_by_max:bool, standardize:bool): 
        # Normalize the dataset
        # I think you should only normalize by max OR standardize, and I think standardizing would produce better results

        # Separate features and target
        df = self.dataframe # make sure data is a data frame
        features = df.drop(['ID', 'label'], axis=1)
        target = df['label']

        df_norm = df # if normalize_by_max = false and standardize = false, will return original df

        if normalize_by_max:
            # normalizing by extremas, scales to [0,1]
            # ensures data is well-conditioned
            features_normalized = (features - features.min())/(features.max() - features.min())
            df_norm = pd.concat([df[['ID']], features_normalized, df[['label']]], axis=1)

        if standardize:
            # z score normalization, good for gaussian distributions
            # forces std 1 and mean 0
            features_standardized = (features - features.mean())/features.std()
            df_norm = pd.concat([df[['ID']], features_standardized, df[['label']]], axis=1)

        # returns a pandas dataframe
        return df_norm
    
    def crossValidation(self, folds:int): 
        # Split dataset into folds
        # I think that self.data should only include non test data
        data = self.data_array[:,1:] # removing first column (ID)
        fold_size = len(data) // folds
        validation_experiments = []
        train_experiments = []

        for i in range(folds):
            if i==(folds-1):
                # how should i deal with uneven split ??? is it okay for the last fold to be smaller?
                validation_fold = data[(i*fold_size):]  #df.iloc[(i*fold_size):(len(df))]
                train_fold = data[:(i*fold_size)] #pd.concat([df.iloc[:(i*fold_size)], df.iloc[(len(df)):]])

            else:
                validation_fold = data[(i*fold_size):(i*fold_size + fold_size)] #df.iloc[(i*fold_size):(i*fold_size + fold_size)]
                train_fold = np.vstack([data[:(i*fold_size)], data[(i*fold_size + fold_size):]]) #pd.concat([df.iloc[:(i*fold_size)], df.iloc[(i*fold_size + fold_size):]])
            validation_experiments.append(validation_fold)
            train_experiments.append(train_fold)

        # Train
        avg_error1 = 0
        avg_error2 = 0
        for i in range(folds):
            # train each training set with fit() to get weights
            train_experiment = train_experiments[i]
            w = self.fit(train_experiment)   

            # get errors 
            validation_experiment = validation_experiments[i]
            error1,error2 = self.Accu_eval(w, validation_experiment) 
            avg_error1 += error1
            avg_error2 += error2

        avg_error1 = avg_error1/folds
        avg_error2 = avg_error2/folds
        return (avg_error1,avg_error2)

    def fit(self,train_data):
        
        # Trains using gradient descent: Lecture 5 slide 55-58

        w_prev = np.ones(train_data.shape[1])*0.1 # (number of columns of train_data - 1 to remove label, + 1 to account for bias term)

        for num_iter in range(1,self.max_iteration+1):
            delta = np.zeros(len(w_prev))

            for row_i in train_data:
                y_i = row_i[-1] # extracting value
                x_i = row_i.copy() # extracting features
                x_i[-1] = 1 # because of bias term
                delta += x_i*(y_i-self.predict(w_prev,x_i))

            
            w_new = w_prev + (self.step_size/num_iter)*delta # to be able to reach better precision need to devide step size by num iteration
       
            w_diff = np.linalg.norm(w_new - w_prev)
            
            w_prev = w_new
            
            if (w_diff**2<self.tolerance):
                return w_new
           
            
        # Means there was a problem
        print("Not converged", w_new , w_diff)
        return w_new

    
    def predict(self,w,x):
        # Predicts output: function at bottom Lecture 5 slide 44
        a = w.T @ x
        return 1/(1+np.exp(-a))

    def Accu_eval(self,w,validate_data): # Used MSE
        # Validation
        error = 0
        correct_prediction = 0
        for row_i in validate_data:
            y_i = row_i[-1] # extracting value
            x_i = row_i.copy() # extracting features
            x_i[-1] = 1
            
            error += (y_i-self.predict(w,x_i))**2 
            if ((y_i-self.predict(w,x_i))<=0.5):
                correct_prediction+=1
            
        return (error/len(validate_data), 1-correct_prediction/len(validate_data))



In [ ]:
# testing whole model for CKD data with cross validation


ckd_model = Model(df_CKD, CKD_data, "This is the model for the CKD dataset")
features_ckd = df_CKD.drop(['ID', 'label'], axis=1)

def condition_number(features):
    A = features.to_numpy()
    U, S, V = np.linalg.svd(A, full_matrices=False)
    condition_num = np.max(S) / np.min(S[np.nonzero(S)])
    return condition_num

# normalize data
df_norm = ckd_model.normalize(True, False)
features_ckd_norm = df_norm.drop(['ID', 'label'], axis=1)
ckd_model_norm = Model(df_norm, df_norm.to_numpy())

# standardize data
df_stand = ckd_model.normalize(False, True)
features_ckd_stand = df_stand.drop(['ID', 'label'], axis=1)
ckd_model_stand = Model(df_stand, df_stand.to_numpy())

print("Condition number of original dataset : ", condition_number(features_ckd))
print("Condition number of normalized dataset : ", condition_number(features_ckd_norm))
print("Condition number of standardized dataset : ", condition_number(features_ckd_stand))
print("---------------------------------------------------------")
print("Error after training original dataset : ", ckd_model.crossValidation(10))
print("Error after training normalized dataset : ", ckd_model_norm.crossValidation(10))
print("Error after training standardized dataset : ", ckd_model_stand.crossValidation(10))


Condition number of original dataset :  1178.1268337574145
Condition number of normalized dataset :  16.272627194785652
Condition number of standardized dataset :  1.6816260377340273
---------------------------------------------------------


C:\Users\issyl\AppData\Local\Temp\ipykernel_30448\2782285286.py:115: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-a))


Error after training original dataset :  (0.26649668292185524, 0.26060606060606056)
Error after training normalized dataset :  (0.2465869549723802, 0.12121212121212119)
Error after training standardized dataset :  (0.21807664123341292, 0.1727272727272727)


In [ ]:
battery_model = Model(df_battery, battery_data, "This is the model for the battery dataset")
features_battery = df_battery.drop(['ID', 'label'], axis=1)

def condition_number(features):
    A = features.to_numpy()
    U, S, V = np.linalg.svd(A, full_matrices=False)
    condition_num = np.max(S) / np.min(S[np.nonzero(S)])
    return condition_num

# normalize data
df_norm = battery_model.normalize(True, False)
features_battery_norm = df_norm.drop(['ID', 'label'], axis=1)
battery_model_norm = Model(df_norm, df_norm.to_numpy())

# standardize data
df_stand = battery_model.normalize(False, True)
features_battery_stand = df_stand.drop(['ID', 'label'], axis=1)
battery_model_stand = Model(df_stand, df_stand.to_numpy())

print("Condition number of original dataset : ", condition_number(features_battery))
print("Condition number of normalized dataset : ", condition_number(features_battery_norm))
print("Condition number of standardized dataset : ", condition_number(features_battery_stand))
print("---------------------------------------------------------")
print("Error after training original dataset : ", battery_model.crossValidation(10))
print("Error after training normalized dataset : ", battery_model_norm.crossValidation(10))
print("Error after training standardized dataset : ", battery_model_stand.crossValidation(10))

Condition number of original dataset :  1178.126833757448
Condition number of normalized dataset :  16.272627194785652
Condition number of standardized dataset :  1.6816260377340286
---------------------------------------------------------


C:\Users\frank\AppData\Local\Temp\ipykernel_47332\2782285286.py:115: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-a))


Error after training original dataset :  (0.26649668292185524, 0.26060606060606056)


In [ ]:
# Adding quadratic features based on weights : features with higher weights will also have a x**2 term

# normalize data
df_ckd_norm = ckd_model.normalize(True, False)
ckd_data_norm = df_ckd_norm.to_numpy()
ckd_model_norm = Model(df_ckd_norm, ckd_data_norm)

train_data = ckd_data_norm[:,1:] # removing first column (ID)
weights = np.array(ckd_model_norm.fit(train_data))
print(weights)

num_quadratic_features = 3
top_weighted_features = np.argsort(np.abs(weights[:-1]))[-num_quadratic_features:] # taking top num_quadratic_features in magnitude of weights, excluding w_0 weight
print(top_weighted_features)

# create new dataset
df_ckd_quadratic = df_CKD.copy()
data_ckd_quadratic = CKD_data.copy()

for i in top_weighted_features:
    df_ckd_quadratic[f'feature_{i}_squared'] = df_ckd_quadratic.iloc[:,i]**2 # adding a new quadratic feature to dataset

# normalize new quadratic dataset
ckd_model_quad = Model(df_ckd_quadratic, data_ckd_quadratic)
df_ckd_quad_norm = ckd_model_quad.normalize(True, False)
ckd_data_quad_norm = df_ckd_quad_norm.to_numpy()
ckd_model_quad_norm = Model(df_ckd_quad_norm, ckd_data_quad_norm)

print(f"Error for model with {num_quadratic_features} quadratic features: ", ckd_model_quad_norm.crossValidation(10))


[-0.05558889  0.15980794  0.00202861  0.00334681 -0.00431049  0.00771148
 -0.00805591  0.00122004 -0.00968128 -0.00588205  0.0267138   0.02610267
  0.00686789  0.00783587 -0.00879119  0.00966012  0.00234501  0.02384556
  0.00453481  0.00237126  0.0209859  -0.0277358  -0.01637442  0.00510092
  0.00684518  0.00283613 -0.00287397  0.01159189 -0.08387512]
[1]
Error for model with 1 quadratic features:  (0.24609957265917637, 0.1484848484848485)


In [ ]:
# Adding cubic features based on weights : features with higher weights will also have a x**3 and x**2 term

# normalize data
df_ckd_norm = ckd_model.normalize(True, False)
ckd_data_norm = df_ckd_norm.to_numpy()
ckd_model_norm = Model(df_ckd_norm, ckd_data_norm)

train_data = ckd_data_norm[:,1:] # removing first column (ID)
weights = np.array(ckd_model_norm.fit(train_data))
print(weights)

num_cubic_features = 3
top_weighted_features = np.argsort(np.abs(weights[:-1]))[-num_cubic_features:] # taking top num_cubic_features in magnitude of weights, excluding w_0 weight
print(top_weighted_features)

# create new dataset
df_ckd_cubic = df_CKD.copy()
data_ckd_cubic = CKD_data.copy()

for i in top_weighted_features:
    df_ckd_cubic[f'feature_{i}_squared'] = df_ckd_cubic.iloc[:,i]**2 # adding a new quadratic feature to dataset
    df_ckd_cubic[f'feature_{i}_cubed'] = df_ckd_cubic.iloc[:,i]**3 # adding a new quadratic feature to dataset

# normalize new cubic dataset
ckd_model_cube = Model(df_ckd_cubic, data_ckd_cubic)
df_ckd_cube_norm = ckd_model_cube.normalize(True, False)
ckd_data_cube_norm = df_ckd_cube_norm.to_numpy()
ckd_model_cube_norm = Model(df_ckd_cube_norm, ckd_data_cube_norm)

print(f"Error for model with {num_cubic_features} cubic features: ", ckd_model_cube_norm.crossValidation(10))

[-0.05558889  0.15980794  0.00202861  0.00334681 -0.00431049  0.00771148
 -0.00805591  0.00122004 -0.00968128 -0.00588205  0.0267138   0.02610267
  0.00686789  0.00783587 -0.00879119  0.00966012  0.00234501  0.02384556
  0.00453481  0.00237126  0.0209859  -0.0277358  -0.01637442  0.00510092
  0.00684518  0.00283613 -0.00287397  0.01159189 -0.08387512]
[10 21  0  1]
Error for model with 4 cubic features:  (0.2458438001748032, 0.19090909090909086)
